# Spam filter
Spam is most commonly associated with emails. For instance, unwanted and unsolicited advertising emails are usually classified as spam. Spamming, however, occurs in ways and environments that don't necessarily relate to emails:

- Articles or blog posts can be spammed with comments — the comments are ads or they are repetitive.
- An educational forum may be spammed with posts that are, in fact, ads.
- Mobile phone users may receive unwanted and unsolicited SMS messages, usually about advertising. 

We are going to build a spam filter specifically directed at preventing mobile phone spam. The filter will be able to analyze new messages and tell whether they are spam or not — this way, we might be able to prevent spam from bothering mobile phone users. Our goal is to create a spam filter that classifies new messages with an accuracy greater than 80% — so we expect that more than 80% of the new messages will be classified correctly as spam or ham (non-spam).

We'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the The [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection).

Note that due to the nature of spam messages, the dataset contains content that may be offensive to some users.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter
from itertools import islice
import re

In [2]:
data = pd.read_table('SMSSpamCollection',header=None, names=['Label', 'SMS'])

### Exploring the dataset

In [3]:
data.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [4]:
data.shape

(5572, 2)

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [6]:
data['Label'].value_counts(normalize=True)

Label
ham     0.865937
spam    0.134063
Name: proportion, dtype: float64

In out dataset we have 86.5% non-spam (ham) messages and 13.5% spam messages.

### Training and Test Set
We're going to keep 80% of our dataset for training, and 20% for testing, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset).

We will randomize the entire dataset to ensure that spam and ham messages are spread properly throughout the dataset.

In [7]:
# Split the data into training and test set, to ensure the class distribution is maintained use stratify parameter

data_train, data_test = train_test_split(
    data,
    test_size=0.2,
    random_state=1,
    stratify=data['Label']
)



In [8]:
data_train = data_train.reset_index(drop=True)
data_train['Label'].value_counts(normalize=True)

Label
ham     0.865829
spam    0.134171
Name: proportion, dtype: float64

In [9]:
data_test = data_test.reset_index(drop=True)
data_test['Label'].value_counts(normalize=True)

Label
ham     0.866368
spam    0.133632
Name: proportion, dtype: float64

The percentage of spam and ham in both the training and the test set are similar to what we have in the full dataset.

### Data Cleaning: Letter Case and Punctuation
Let's begin the data cleaning process by removing the punctuation and bringing all the words to lower case.

In [10]:
# Remove all punctuation from SMS column
data_train['SMS'] = data_train['SMS'].str.replace(r'\W', ' ', regex=True).str.lower()
data_train.head()


,Label,SMS
0,ham,no he joined today itself
1,ham,will ü b going to esplanade fr home
2,spam,reply to win 100 weekly what professional sp...
3,ham,awesome lemme know whenever you re around
4,spam,private your 2003 account statement for shows...


In [11]:
data_test.head()

,Label,SMS
0,ham,Ok i shall talk to him
1,ham,Eatin my lunch...
2,ham,"Sir, Waiting for your mail."
3,ham,Wif my family booking tour package.
4,ham,No. It's not pride. I'm almost &lt;#&gt; yea...


###  Creating the Vocabulary
Let's create a list with all of the unique words that occur in the messages of our training set.

In [12]:
words = Counter()
data_train['sms_words'] = data_train['SMS'].str.strip().str.split('\s+')
data_train['sms_words'] .apply(words.update)
vocab = list(words.keys())
vocab[:10]

['no', 'he', 'joined', 'today', 'itself', 'will', 'ü', 'b', 'going', 'to']

### The Final Training Set
We need a transformed table that represents a unique word in our vocabulary where each column shows the frequency of that unique word for any given message.


In [13]:
vocab_dict = {word:[0]*len(data_train) for word in vocab}

for i,sms in enumerate(data_train['sms_words']):
    for word in sms:
        vocab_dict[word][i] += 1


In [14]:
df = pd.DataFrame(vocab_dict)
df.head()

,no,he,joined,today,itself,will,ü,b,going,to,...,yowifes,hint,helens,princes,arguing,gailxx,betta,aging,products,badrith
0,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
df.shape

(4457, 7671)

In [16]:
final_df = pd.concat([data_train, df], axis=1)
final_df.head()

,Label,SMS,sms_words,no,he,joined,today,itself,will,ü,...,yowifes,hint,helens,princes,arguing,gailxx,betta,aging,products,badrith
0,ham,no he joined today itself,"[no, he, joined, today, itself]",1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,will ü b going to esplanade fr home,"[will, ü, b, going, to, esplanade, fr, home]",0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,spam,reply to win 100 weekly what professional sp...,"[reply, to, win, 100, weekly, what, profession...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,awesome lemme know whenever you re around,"[awesome, lemme, know, whenever, you, re, around]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,spam,private your 2003 account statement for shows...,"[private, your, 2003, account, statement, for,...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Calculating Constants

Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter. 

Naive Bayes algorithm will make the classification based on the results it gets to these two equations:
$$P(Spam|w1,w2,...,wn)∝P(Spam)⋅ \prod_{i=1}^n P(wi|Spam)$$
$$P(Ham|w1,w2,...,wn)∝P(Ham)⋅ \prod_{i=1}^n P(wi|Ham)$$

To calculate P(wi|Spam) and P(wi|Ham) inside the formulas above, we need to use these equations:
$$P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}$$
$$P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}$$

Where:   
$N_{w_i|Spam}$=the number of times the word wi occurs in spam messages   
$N_{w_i|Ham}$=the number of times the word wi occurs in non-spam messages   
$N_{Spam}$=total number of words in spam messages   
$N_{Ham}$=total number of words in non-spam messages   
$N_{Vocabulary}$=total number of words in the vocabulary   
$α=1$    (α is a smoothing parameter)


In [17]:
# total number of words in spam messages
n_spam = final_df.loc[final_df.Label == 'spam','sms_words'].apply(len).sum()
# total number of words in ham messages
n_ham = final_df.loc[final_df.Label == 'ham','sms_words'].apply(len).sum()
# total number of words in vocabulary
n_vocab = len(vocab)
# smoothing parameter set to 1 for Laplace smoothing
alpha = 1

proprotion = final_df['Label'].value_counts(normalize=True)
p_spam = proprotion['spam']
p_ham = proprotion['ham']

print(f'p_ham = {p_ham}, p_spam = {p_spam}, n_spam = {n_spam}, n_ham = {n_ham}, n_vocab = {n_vocab}, alpha = {alpha}')

p_ham = 0.8658290329818263, p_spam = 0.13417096701817366, n_spam = 15290, n_ham = 56729, n_vocab = 7671, alpha = 1


### Calculating Parameters

Now let's calculate the parameters using these equations:
$$P(w_i|Spam) = \frac{N_{w_i|Spam} + \alpha}{N_{Spam} + \alpha \cdot N_{Vocabulary}}$$
$$P(w_i|Ham) = \frac{N_{w_i|Ham} + \alpha}{N_{Ham} + \alpha \cdot N_{Vocabulary}}$$

We will create two dictionaries - one for spam and ham each. The keys will be the words and values will be $P(w_i|Spam)$ and $P(w_i|Ham)$ respectively.

In [18]:
# initialize dictionaries with 0 value for each word in vocab
p_wi_spam = {word:0 for word in vocab}
p_wi_ham = {word:0 for word in vocab}


spam_only = final_df[final_df.Label == 'spam']
ham_only = final_df[final_df.Label == 'ham']

n_wi_spam = spam_only.iloc[:, 3:].sum()
n_wi_ham = ham_only.iloc[:, 3:].sum()

In [19]:
n_wi_spam.head(10)

no         52
he          0
joined      2
today      18
itself      0
will       37
ü           0
b           9
going       3
to        546
dtype: int64

In [20]:
# calculate p_wi_spam
for word,v in p_wi_spam.items():
    p_wi_spam[word] = (n_wi_spam[word] + alpha)/(n_spam + (alpha*n_vocab))
    
for k,v in islice(p_wi_spam.items(),10):
    print(k,v)


no 0.0023082618352859197
he 4.3552110099734335e-05
joined 0.000130656330299203
today 0.0008274900918949523
itself 4.3552110099734335e-05
will 0.0016549801837899046
ü 4.3552110099734335e-05
b 0.0004355211009973433
going 0.00017420844039893734
to 0.02382300422455468


In [21]:
# calculate p_wi_ham
for word,v in p_wi_spam.items():
    p_wi_ham[word] = (n_wi_ham[word] + alpha)/(n_ham + (alpha*n_vocab))

for k,v in islice(p_wi_ham.items(),10):
    print(k,v)

no 0.0036645962732919255
he 0.0026086956521739132
joined 9.316770186335404e-05
today 0.0018167701863354038
itself 0.00015527950310559007
will 0.004239130434782609
ü 0.00218944099378882
b 0.0009472049689440994
going 0.0020496894409937887
to 0.019580745341614907


### Classifying a new message
Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter will be a function that:

- Takes in as input a new message (w1, w2, ..., wn). 
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.

Where:
$$P(Spam|w1,w2,...,wn)∝P(Spam)⋅ \prod_{i=1}^n P(wi|Spam)$$
$$P(Ham|w1,w2,...,wn)∝P(Ham)⋅ \prod_{i=1}^n P(wi|Ham)$$

Note that some new messages will contain words that are not part of the vocabulary. We will ignore these words when we're calculating the probabilities.

In [22]:
# The input variable message is assumed to be a string

def classify(sms):
    sms = re.sub('\W', ' ', sms) # replace all punctuation with a single space
    sms = re.sub('\s+', ' ', sms) # replace multiple spaces with a single space
    sms = sms.strip().split(' ') #split words into a list

    p_spam_given_sms = p_spam
    p_ham_given_sms = p_ham
    for word in sms:
        if word in vocab:
            p_spam_given_sms *= p_wi_spam[word]
            p_ham_given_sms *= p_wi_ham[word]
    #print(p_spam_given_sms, p_ham_given_sms)
    if p_spam_given_sms > p_ham_given_sms:
        return 'spam'
    elif p_ham_given_sms > p_spam_given_sms:
        return 'ham'
    else:
        return 'needs human classification'



In [23]:
test1 = 'WINNER!! This is the secret code to unlock the money: C3421.'
classify(test1)

'spam'

In [24]:
test2 = 'Sounds good, Tom, then see u there'
classify(test2)

'ham'

### Measuring the Spam Filter's Accuracy
We managed to create a spam filter, and we classified two new messages. We'll now try to determine how well the spam filter does on our test set of 1,115 messages.

Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric:

$$\text{Acurracy} = \frac{{\text{number of correctly classified messages}}} {\text{total number of classified messages}}$$

In [25]:
data_test['predicted'] = data_test['SMS'].apply(classify)
data_test.head()

,Label,SMS,predicted
0,ham,Ok i shall talk to him,ham
1,ham,Eatin my lunch...,ham
2,ham,"Sir, Waiting for your mail.",ham
3,ham,Wif my family booking tour package.,ham
4,ham,No. It's not pride. I'm almost &lt;#&gt; yea...,ham


In [26]:
data_test['predicted'].value_counts()

predicted
ham     975
spam    140
Name: count, dtype: int64

In [27]:
(data_test['Label'] != data_test['predicted']).sum()

19

In [28]:
n_correct = (data_test['Label'] == data_test['predicted']).sum()
n_total = data_test.shape[0]
accuracy = n_correct*100/n_total
accuracy

98.29596412556054

Filter has an accuracy of 98.29%.

## Conclusion

In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter has an accuracy of 98.29% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.

Next Steps:
Filter classified 19 sms incorrectly and we can improve the accuracy further by making the algorithm sensitive to letter case.

### Author
Puneet Pawar